# Genetic Programing 2: Evaluation and Mutation

### This is the code from the last notebook, just combined into a single Jupyter cell

In [1]:
import numpy as np
from matplotlib import pyplot
import random

class GPNode:
    def __init__(self, node_type=None):
        self.parent = None
        self.node_type = node_type
        self.children = []
        self.depth = 0
        
    def add_child(self, child_node):
        child_node.depth = self.depth+1
        self.children.append(child_node)
        child_node.parent = self
    
    
class GPConstNode(GPNode):
    def __init__(self, value=None):
        super().__init__(node_type="Const")
        self.const_value = value
    
    def evaluate(self, input_state):
        return self.const_value
        
    def pretty_print(self, indents=0):
        print('  '*indents + str(self.const_value) 
              + ' : ' + str(self.depth))
        
    def deepcopy(self):
        new_node = GPConstNode(value=self.const_value)
        new_node.depth = self.depth;
        return new_node
        
        
class GPVariableNode(GPNode):
    def __init__(self, variable_name=None):
        super().__init__(node_type="Variable")
        self.variable_name = variable_name
    
    def evaluate(self, input_state):
        return input_state[self.variable_name]
    
    def pretty_print(self, indents=0):
        print('  '*indents + str(self.variable_name)
              + ' : ' + str(self.depth))
        
    def deepcopy(self):
        new_node = GPVariableNode(variable_name = self.variable_name)
        new_node.depth = self.depth
        return new_node
        
class GPFunctionNode(GPNode):
    def __init__(self, arg_count, func_name=None, gp_function=None):
        super().__init__(node_type="Function")
        self.argument_count = arg_count
        self.gp_function = gp_function
        self.function_name = func_name
        
    def evaluate(self, input_state): 
        assert self.argument_count == len(self.children), \
        'Number of child nodes must match argument count'

        child_results = [c.evaluate(input_state) for c in self.children]
        return self.gp_function(*child_results)

    def pretty_print(self, indents=0):
        print('  '*indents + str(self.function_name) 
              + ' : ' + str(self.depth))
        
        for child in self.children:
            child.pretty_print(indents+1)
        
    def deepcopy(self):
        new_node = GPFunctionNode(self.argument_count, 
                                   self.function_name, 
                                   self.gp_function)
        new_node.depth = self.depth
        
        for child in self.children:
            new_node.add_child(child.deepcopy())
        
        return new_node


# Here we're defining an individual organism

In [2]:
class GPIndividual:
    # This is the beef of the individual code. We're growing random 
    # trees with a bit of extra sauce. 
    # We've defined a prob_terminal value that determines how 
    # likely it is that we select a terminal when choosing a random GP node. 
    # We're also limiting the depth of the trees we grow, because this is 
    # Python after all. 
    def grow_random(self, cur_node=None, cur_depth=0):
        if (random.random() < self.prob_terminal 
            or cur_depth == self.max_genotype_depth-1):
            new_node = random.choice(self.terminal_set).deepcopy()
            new_node.parent = cur_node
            new_node.depth = cur_depth
        else:
            new_node = random.choice(self.function_set).deepcopy()
            new_node.depth = cur_depth
            new_node.parent = cur_node
            for i in range(new_node.argument_count):
                new_node.add_child(self.grow_random(cur_node=new_node, 
                                                    cur_depth=cur_depth+1))
        return new_node
    
    
    # prob_terminal and max_depth have default parameters but you 
    # can use them to control how big the trees can get and how
    # likely you are to pick function/terminal nodes when growing
    # and mutating trees. 
    def __init__(self, function_set, terminal_set, 
                 prob_terminal=0.2, max_depth=5):
        self.max_genotype_depth = max_depth
        self.fitness = None
        self.function_set = function_set
        self.terminal_set = terminal_set
        self.prob_terminal = prob_terminal
        
        self.genotype = self.grow_random()
        
        
    def pretty_print(self):
        self.genotype.pretty_print()
        
        
    def deepcopy(self):
        new_individual = GPIndividual(self.function_set, 
                                      self.terminal_set, 
                                      self.prob_terminal, 
                                      self.max_genotype_depth)
        new_individual.genotype = self.genotype.deepcopy()
        return new_individual
    
    
    # This function just visits each node in the genome, growing
    # the list of nodes as it encounters children.
    def visit_genotype_nodes(self, cur_node=None):
        if cur_node == None: 
            cur_node = self.genotype
            
        node_list = [cur_node]
        visitor_index = 0
        
        while visitor_index < len(node_list):
            if len(node_list[visitor_index].children) > 0:
                node_list.extend(node_list[visitor_index].children)
            visitor_index += 1
            
        return node_list
        
    
    def evaluate(self, input_state):
        #evaluate the genotype
        return self.genotype.evaluate(input_state)
        
        
    def mutate(self):
        # get a list of nodes...
        genotype_nodes = self.visit_genotype_nodes()
        # and pick one! 
        random_node = random.choice(genotype_nodes)
        
        if random_node.parent == None:
            #We've picked the root, so just grow a whole new genotype
            self.genotype = self.grow_random()
        else:
            #generate a new subtree using the random node's parent
            #as the parent for this subtree
            new_node = self.grow_random(random_node.parent, random_node.depth)
            #remove old node, add new node to parent's list of children
            random_node.parent.children.remove(random_node)
            random_node.parent.add_child(new_node)
            

## To build an individual, we need a set of function nodes and a set of terminal nodes. Let's start simple with the ones we worked on last class. 
Remember that the **`lambda`** keyword in this code is telling python that we want to create a function, but we don't really care what it's called. 

In [3]:
gp_add = GPFunctionNode(arg_count=2, func_name="Add", gp_function=lambda x, y: x+y)
gp_sub = GPFunctionNode(arg_count=2, func_name="Sub", gp_function=lambda x, y: x-y)

gp_const1 = GPConstNode(42)
gp_const2 = GPConstNode(3.14)
gp_var1 = GPVariableNode(variable_name='x')


gp_func_set = [gp_add, gp_sub]
gp_term_set = [gp_const1, gp_const2, gp_var1]

### Now that we have a funciton set and a terminal set, we can build an individual.

In [5]:
gp_ind = GPIndividual(function_set=gp_func_set, terminal_set=gp_term_set)
gp_ind.pretty_print()

Sub : 0
  Sub : 1
    Sub : 2
      Sub : 3
        x : 4
        x : 4
      Sub : 3
        x : 4
        42 : 4
    x : 2
  Sub : 1
    Sub : 2
      Add : 3
        42 : 4
        3.14 : 4
      Sub : 3
        42 : 4
        3.14 : 4
    Add : 2
      Add : 3
        3.14 : 4
        42 : 4
      Add : 3
        3.14 : 4
        x : 4


### And we can evaluate it our newly grown random individual, remembering to pass in a state value for x!

In [6]:
gp_ind.evaluate({'x':2.5})

81.5

# Now we will build a class that holds a whole population of individuals, and can evaluate them against some fitness function we choose, and runs an evolutionary simulation like we're so good at now 

### `update_fitness(fitness_function)`
This function calls a passed in function that we haven't defined yet (**fitness_function**) and calls it on each individual in the population. It takes the output of that fitness_function and stores it in the individual's fitness variable (e.g., gp_ind.fitness).

### `do_timestep(fitness_function, selection_function, mutation_prob)`
This function uses the **fitness_function** to update each individual's fitness value, and then calls the **selection_function**, with the full population of individuals as a parameter, to pick an organism to reproduce. It calls this function **pop_size** number of times to fill up the population for the next generation. Each individual has some chance of mutating, which is what the **mutation_prob** is parameterizing. 

In [20]:
class GPPopulation:
    def __init__(self, pop_size, function_set, terminal_set, max_depth, prob_terminal):
        self.pop_size = pop_size
        self.terminal_set = terminal_set
        self.function_set = function_set
        self.max_depth = max_depth
        self.prob_terminal = prob_terminal
        
        self.population = [GPIndividual(self.function_set, 
                                        self.terminal_set,
                                        self.prob_terminal, 
                                        self.max_depth) 
                           for _ in range(self.pop_size)]
        
        
    def update_fitnesses(self, fitness_function):
        fitnesses = []
        for individual in self.population:
            individual.fitness = fitness_function(individual)
            fitnesses.append(individual.fitness)
            
        return fitnesses
    
    
    def do_timestep(self, fitness_function, selection_function, mutation_prob):
        fitness_list = self.update_fitnesses(fitness_function)
        selected_individuals = [selection_function(self.population) 
                                for _ in range(self.pop_size)]
        
        for individual_idx in range(len(selected_individuals)):
            individual = selected_individuals[individual_idx].deepcopy()
            
            if random.random() < mutation_prob:
                individual.mutate()
            
            selected_individuals[individual_idx] = individual
        
        self.population = selected_individuals
        
        return fitness_list
        

## In order for this code to work, you need to implement a `fitness_function` that evaluates an GPIndividual and returns a fitness value. You'll also need to implement a `selection_function` that is passed the population of individuals (where each individual now has a fitness value assigned to it), and returns a single individual that gets to take a spot in the next generation.

# 1. Write a `fitness_function` that evaluates the expression tree. A good place to start is defining the fitness as just the value the tree evaluates to (though make sure fitness is positive!).  

In [21]:
def my_fitness_function(gp_individual):
    #IMPLEMENT THIS
    
    return 1.0

# 2. Write a `selection_function` that implements tournament selection. Try a tournament of size 5 for now.

In [22]:
def my_selection_function(gp_population):
    #IMPLEMENT THIS
    
    return random.choice(gp_population)

#### We can create our population now

In [23]:
gp_pop = GPPopulation(pop_size=100, 
                      function_set=gp_func_set, 
                      terminal_set=gp_term_set,
                      max_depth=6,
                      prob_terminal=0.2)


### And when you have implemeneted your fitness function and selection functions, you should be able to run this cell multiple times and see the population change. The `do_timestep` function returns a list of the fitnesses of the individuals in the population. You can use that to keep track of the mean fitness, and plot it, for example. 

In [25]:
gp_pop.do_timestep(fitness_function=my_fitness_function, 
                   selection_function=my_selection_function,
                   mutation_prob=0.1)


[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

# 3. Write a loop to iterate the `do_timestep` call for a number of generations, and plot the mean fitness values.

# 4. Write a more interesting fitness function, perhaps give the genetic program something as a goal that is too hard for it's current function nodes -- like sin(x). 

# 5. Add in some function nodes that might make for more fun solutions to #4, but don't give it the answer (e.g., don't add a `sin` or `cos` function node!).